# Deployment

Now that we've designed a simple retrieval chain, let's look at what it would take to deploy it as a streaming chat endpoint!

We'll pick up where we left off in the last lesson with loading and splitting our CS229 transcript into a vectorstore:

In [1]:
import "npm:dotenv/config";

[Module: null prototype] { default: {} }

In [2]:
// Peer dependency
import * as parse from "npm:pdf-parse";
import { PDFLoader } from "npm:langchain@0.0.202/document_loaders/fs/pdf";
import { RecursiveCharacterTextSplitter } from "npm:langchain@0.0.202/text_splitter";
import { MemoryVectorStore } from "npm:langchain@0.0.202/vectorstores/memory";
import { OpenAIEmbeddings } from "npm:langchain@0.0.202/embeddings/openai";

const loader = new PDFLoader("./static/docs/MachineLearning-Lecture01.pdf");

const rawCS229Docs = await loader.load();

const splitter = new RecursiveCharacterTextSplitter({
  chunkSize: 1536,
  chunkOverlap: 128,
});

const splitDocs = await splitter.splitDocuments(rawCS229Docs);

const embeddings = new OpenAIEmbeddings();

const vectorstore = new MemoryVectorStore(embeddings);

await vectorstore.addDocuments(splitDocs);

Creating a retriever from the vectorstore:

In [3]:
const retriever = vectorstore.asRetriever();

And finally, putting the pieces of our conversational retrieval chain together.

In [4]:
import { RunnableSequence } from "npm:langchain@0.0.202/runnables";

const convertDocsToString = (documents: Document[]): string => {
  return documents.map((document) => `<doc>\n${document.pageContent}\n</doc>`).join("\n");
};

const documentRetrievalChain = RunnableSequence.from([
  (input) => input.standalone_question,
  retriever,
  convertDocsToString,
]);

In [5]:
import { ChatOpenAI } from "npm:langchain@0.0.202/chat_models/openai";
import { StringOutputParser } from "npm:langchain@0.0.202/schema/output_parser";
import { ChatPromptTemplate, MessagesPlaceholder } from "npm:langchain@0.0.202/prompts";

const REPHRASE_QUESTION_SYSTEM_TEMPLATE = `Using the provided chat history as context, rephrase the following question to be a standalone question that has no external references.

Do not respond with anything other than a rephrased standalone question.`;

const rephraseQuestionChainPrompt = ChatPromptTemplate.fromMessages([
  ["system", REPHRASE_QUESTION_SYSTEM_TEMPLATE],
  new MessagesPlaceholder("history"),
  ["human", "Now, answer the following question:\n{question}"],
]);

const rephraseQuestionChain = RunnableSequence.from([
  rephraseQuestionChainPrompt,
  new ChatOpenAI({ temperature: 0, modelName: "gpt-3.5-turbo-1106" }),
  new StringOutputParser(),
]);

In [6]:
const ANSWER_CHAIN_SYSTEM_TEMPLATE = `You are an experienced researcher, expert at interpreting and answering questions based on provided sources.
Using the below provided context and chat history, answer the user's question to the best of your ability using only the resources provided. Be concise!

<context>
{context}
</context>`;

const answerGenerationChainPrompt = ChatPromptTemplate.fromMessages([
  ["system", ANSWER_CHAIN_SYSTEM_TEMPLATE],
  new MessagesPlaceholder("history"),
  ["human", "Now, answer this question using the previous context and chat history:\n{standalone_question}"]
]);


Before we assemble all the pieces together, let's note that web `Response` objects that you return in popular frameworks like Next.js accept a `ReadableStream` the emits bytes directly. Previously, our chain outputted string chunks directly using `StringOutputParser`, but it would be convenient to be able to directly stream so that we could pass our LangChain stream directly to the response.

Fortunately, LangChain provides an `HttpResponseOutputParser` that parses chat output into chunks of bytes that match either `text/plain` or `text/event-stream` content types! To use it, let's construct our conversational retrieval chain as before, but skip the final `StringOutputParser`:

In [ ]:
import { RunnablePassthrough } from "npm:langchain@0.0.202/runnables";

const conversationalRetrievalChain = RunnableSequence.from([
  RunnablePassthrough.assign({
    standalone_question: rephraseQuestionChain,
  }),
  RunnablePassthrough.assign({
    context: documentRetrievalChain,
  }),
  answerGenerationChainPrompt,
  new ChatOpenAI({ modelName: "gpt-3.5-turbo" }),
]);

Then, we'll create an `HttpResponseOutputParser` and pipe the `RunnableWithMessageHistory` into it:

In [7]:
import { HttpResponseOutputParser } from "npm:langchain@0.0.202/output_parsers";
import { RunnableWithMessageHistory } from "npm:langchain@0.0.202/runnables"; 
import { ChatMessageHistory } from "npm:langchain@0.0.202/memory";

// "text/event-stream" is also supported
const httpResponseOutputParser = new HttpResponseOutputParser({
  contentType: "text/plain"
});

const messageHistory = new ChatMessageHistory();

const conversationalRetrievalChainWithHistory = new RunnableWithMessageHistory({
  runnable: conversationalRetrievalChain,
  // Mention where sessionId gets passed from (parameter to our endpoint)
  getMessageHistory: (_sessionId) => messageHistory,
  inputMessagesKey: "question",
  historyMessagesKey: "history",
}).pipe(httpResponseOutputParser);

The reason we don't put the `HttpResponseOutputParser` directly in the `conversationalRetrievalChain` is because `RunnableWithMessageHistory` will store the aggregated output of its runnable in the `ChatMessageHistory`, and requires either a string or a `ChatMessage` to be the final output rather than bytes.

You might also notice that our `getMessageHistory` function creates a new `RedisMessageHistory` object instead of reusing an in memory `ChatMessageHistory` as before. This allows us to assign `sessionId`s properly to individual conversations and load them as requests come in later.

Great! Let's set up a simple server with a handler that calls our chain and see if we can get a streaming response. We'll populate the input question and the session id from the body paramters. Since this notebook is written in Deno, we use a Deno built-in HTTP method, but this general concept is shared by many JS frameworks.

Also, in a true production deployment, you'd likely want to set up authentication/input validation, but we'll skip that for simplicity for now:

In [8]:
const port = 8080;

const handler = async (request: Request): Response => {
  const body = await request.json();
  const stream = await conversationalRetrievalChainWithHistory.stream({
    question: body.question
  }, { configurable: { sessionId: body.session_id } });

  return new Response(stream, { 
    status: 200,
    headers: {
      "Content-Type": "text/plain"
    }
  });
};

console.log(`HTTP server is running! Access it at: http://localhost:${port}/`);
Deno.serve({ port }, handler);

HTTP server is running! Access it at: http://localhost:8080/
Listening on http://localhost:8080/


{
  finished: Promise { <pending> },
  shutdown: [AsyncFunction: shutdown],
  ref: [Function: ref],
  unref: [Function: unref]
}

Let's make a quick helper function to make handling the response stream in the client a bit nicer:

In [12]:
const decoder = new TextDecoder();

// readChunks() reads from the provided reader and yields the results into an async iterable
function readChunks(reader) {
  return {
    async* [Symbol.asyncIterator]() {
      let readResult = await reader.read();
      while (!readResult.done) {
        yield decoder.decode(readResult.value);
        readResult = await reader.read();
      }
    },
  };
}

And now let's try calling our endpoint!

In [9]:
const response = await fetch("http://localhost:8080/", {
  method: "POST",
  headers: {
    "content-type": "application/json",
  },
  body: JSON.stringify({
    question: "What are the prerequisites for this course?",
    session_id: "1", // Should randomly generate/assign
  })
});

// response.body is a ReadableStream
const reader = response.body?.getReader();

for await (const chunk of readChunks(reader)) {
  console.log("CHUNK:", chunk);
}

CHUNK: The req
CHUNK: uirements for
CHUNK:  this course 
CHUNK: are familiarity wit
CHUNK: h basic probabil
CHUNK: ity and statistics, basic linear algebra, and some programming experience in MATLAB or Octave. Familiarity with random variables, expectation, variance, matrices, vectors, matrix multiplication, matrix inverse, and eigenvectors is also beneficial but not mandatory.


We can see that we get a streamed string response that we could render in a frontend web component!

Now, let's test the memory by asking a followup:

In [1]:
const response = await fetch("http://localhost:8080/", {
  method: "POST",
  headers: {
    "content-type": "application/json",
  },
  body: JSON.stringify({
    question: "What did I just ask you?",
    session_id: "1", // Should randomly generate/assign
  })
});

// response.body is a ReadableStream
const reader = response.body?.getReader();

for await (const chunk of readChunks(reader)) {
  console.log("CHUNK:", chunk);
}

TypeError: error sending request for url (http://localhost:8080/): error trying to connect: tcp connect error: Connection refused (os error 61)

Sweet! Let's try again with a different `sessionId`. We expect to see a wholly new loaded conversation.